# Imports

In [4]:
from pathlib import Path
import sys
import warnings

import pandas as pd

abs_path = Path().resolve()
sys.path.append(str(abs_path.parent / "modules"))

from files_funcs import *
from docx_funcs import *


warnings.filterwarnings("ignore")

# Config

In [9]:
main_dir = abs_path.parent.parent

config_path = abs_path.parent.parent / "config" / "config.yaml"
config = load_yaml(config_path)

data_dir = main_dir / "data"
uc_data_dir = data_dir / "raw_data" / "HMI"
ssts_data_dir = data_dir / "raw_data" / "SSTS"

columns = (
    config["ds_topology"]["main_columns"]
    + [f'{col}' for col in set(config["ds_topology"]["hmi_subheaders"].values())]
    + config["ds_topology"]["targets"]
)
columns = [col.lower().replace(" ", "_") for col in columns]

# Loading

In [10]:
hmi_files = [file.name for file in uc_data_dir.iterdir() if file.is_file()]
ssts_files = [file.name for file in ssts_data_dir.iterdir() if file.is_file()]

ids = set(
    [
        "".join(char for char in fname if char.isdigit())
        for fname in (
            hmi_files + ssts_files
        )
    ]
)

ds_rows = [
    create_ds_row(fid, uc_data_dir, ssts_data_dir, columns)
    for fid in ids
]

ds = pd.DataFrame(ds_rows, columns=columns)
ds

Warning! [Errno 2] No such file or directory: 'C:\\Users\\yytrb\\OneDrive\\Рабочий стол\\atom-compliance-ml\\data\\raw_data\\SSTS\\SSTS-26160.docx'


,id,case_name,full_uc_text,full_ssts_text,other,preconditions,goal,postconditions,main_scenario,differences,description,complience_level
0,30371,ERA Self-diagnosis,[I-30371] ERA Self-diagnosis\nActors:\nERA-Gl...,ERA Self-diagnosis\nFunctional Description\n\n...,Description:\nERA self-diagnosis whether the s...,,,,,None,None,None
1,8692,Emergency Service Communication (ERA-Glonass),[I-8692] Emergency Service Communication (ERA...,Manual dialing E-CALL\nFunctional Description\...,Trigger:\nThe vehicle sensors detect a collisi...,Preconditions:\nThe vehicle is in the Drive St...,,Postconditions：\nThe driver is able to communi...,Main Scenario:\nThe ERA-Glonass initiates emer...,None,None,None
2,6583,Driver initiate a call through SWP,[I-6583] Driver initiate a call through SWP\n...,Make a call (B sample)\nFunctional Description...,,Preconditions:\nThe IVI is on in_2 \nThe vehic...,,Postconditions:\nIf the call is initiated succ...,Main Scenario:\nThe driver navigates to the 'C...,None,None,None
3,8800,Receiving Call Notifications,[I-8800] Receiving Call Notifications\nPrecon...,Receiving Call Notifications\nFunctional Descr...,,Preconditions\nEntertainment system is ready. ...,,"Postconditions:\nIf the call is accepted, the ...",Main Scenario:\nAn incoming call is received o...,None,None,None
4,11467,Revoke access to the vehicle from a driver or ...,[I-11467] Revoke access to the vehicle from a...,Users can remotely control the DK deletion thr...,,Preconditions:\nAlice - the owner or the drive...,Goal:\nEnable the Owner to quickly revoke acce...,Postconditions:\nAccess has been successfully ...,Main Scenario:\nAlice opens the ATOM applicati...,None,None,None
5,26161,FM Radio Stations switching,[I-26161] FM Radio Stations switching\nDescri...,Automatic search\nFunctional Description\nSupp...,Description: Requirenments:\nThe action is dis...,Preconditions:\nUser has selected FM Radio pla...,,Postconditions:\nout_5/out_2 shows the station...,Main Scenario:\nUser presses the forward/backw...,None,None,None
6,26771,Turn on and off hotspot via VA,[I-26771] Turn on and off hotspot via VA\n\nD...,\nTurn on and off hotspot\nFunctional Descript...,Description: Scope:\nSWP Android \nVA \nSmartp...,,Goal: \nUsers can turn on or off the vehicle h...,,Main scenario:\nThe user asks in_24 Voice. Mic...,None,None,None
7,28561,Setting Hotspot name & password,[I-28561] Setting Hotspot name & password\nUs...,hotspot settings\nFunctional Description\nUser...,Scope: \nSWP Actors:\nDriver \nOwner Requireme...,,Goal:\nUsers can modify the name and password ...,,Main Scenario:\nThe user guides to the hotspot...,None,None,None
8,8604,Source selection for Media output,[I-8604] Source selection for Media output\nD...,Switch music sources\nFunctional Description\n...,Description: Scope: This use case is about sel...,Preconditions:\nATOM is on \nin_5. Smartphone ...,Goal: Enable user selection of multimedia audi...,Postconditions:\nout_2/out_5 shows available m...,"Main Scenario:\nUser selects the ""Multimedia"" ...",None,None,None
9,25957,Mute/unmute the FM Radio playback,[I-25957] Mute/unmute the FM Radio playback\n...,Mute or pause function\nFunctional Description...,Description:,Preconditions:\nUser is playing FM Radio,"Goal: ""Users are able to mute/unmute the FM Ra...",Postconditions:\nThe user Mutes/Unmutes the FM...,Main Scenario:\n​​​User clicks on Mute/Unmute ...,None,None,None


# Target

In [11]:
marked_data = pd.read_excel(data_dir / "raw_data" / "train_data_markup.xlsx")
marked_data = marked_data[["Number", "Complience Level"]]
marked_data = marked_data.rename(columns={"Number": "id", "Complience Level": "complience_level"})
marked_data["target"] = marked_data["complience_level"].fillna("NA").map(config["target_dict"])
marked_data = marked_data.drop("complience_level", axis=1)
marked_data["id"] = marked_data["id"].astype(str)

ds = ds.merge(marked_data, on="id")
ds

,id,case_name,full_uc_text,full_ssts_text,other,preconditions,goal,postconditions,main_scenario,differences,description,complience_level,target
0,30371,ERA Self-diagnosis,[I-30371] ERA Self-diagnosis\nActors:\nERA-Gl...,ERA Self-diagnosis\nFunctional Description\n\n...,Description:\nERA self-diagnosis whether the s...,,,,,None,None,None,4
1,8692,Emergency Service Communication (ERA-Glonass),[I-8692] Emergency Service Communication (ERA...,Manual dialing E-CALL\nFunctional Description\...,Trigger:\nThe vehicle sensors detect a collisi...,Preconditions:\nThe vehicle is in the Drive St...,,Postconditions：\nThe driver is able to communi...,Main Scenario:\nThe ERA-Glonass initiates emer...,None,None,None,4
2,6583,Driver initiate a call through SWP,[I-6583] Driver initiate a call through SWP\n...,Make a call (B sample)\nFunctional Description...,,Preconditions:\nThe IVI is on in_2 \nThe vehic...,,Postconditions:\nIf the call is initiated succ...,Main Scenario:\nThe driver navigates to the 'C...,None,None,None,2
3,8800,Receiving Call Notifications,[I-8800] Receiving Call Notifications\nPrecon...,Receiving Call Notifications\nFunctional Descr...,,Preconditions\nEntertainment system is ready. ...,,"Postconditions:\nIf the call is accepted, the ...",Main Scenario:\nAn incoming call is received o...,None,None,None,4
4,11467,Revoke access to the vehicle from a driver or ...,[I-11467] Revoke access to the vehicle from a...,Users can remotely control the DK deletion thr...,,Preconditions:\nAlice - the owner or the drive...,Goal:\nEnable the Owner to quickly revoke acce...,Postconditions:\nAccess has been successfully ...,Main Scenario:\nAlice opens the ATOM applicati...,None,None,None,5
5,26161,FM Radio Stations switching,[I-26161] FM Radio Stations switching\nDescri...,Automatic search\nFunctional Description\nSupp...,Description: Requirenments:\nThe action is dis...,Preconditions:\nUser has selected FM Radio pla...,,Postconditions:\nout_5/out_2 shows the station...,Main Scenario:\nUser presses the forward/backw...,None,None,None,4
6,26771,Turn on and off hotspot via VA,[I-26771] Turn on and off hotspot via VA\n\nD...,\nTurn on and off hotspot\nFunctional Descript...,Description: Scope:\nSWP Android \nVA \nSmartp...,,Goal: \nUsers can turn on or off the vehicle h...,,Main scenario:\nThe user asks in_24 Voice. Mic...,None,None,None,4
7,28561,Setting Hotspot name & password,[I-28561] Setting Hotspot name & password\nUs...,hotspot settings\nFunctional Description\nUser...,Scope: \nSWP Actors:\nDriver \nOwner Requireme...,,Goal:\nUsers can modify the name and password ...,,Main Scenario:\nThe user guides to the hotspot...,None,None,None,4
8,8604,Source selection for Media output,[I-8604] Source selection for Media output\nD...,Switch music sources\nFunctional Description\n...,Description: Scope: This use case is about sel...,Preconditions:\nATOM is on \nin_5. Smartphone ...,Goal: Enable user selection of multimedia audi...,Postconditions:\nout_2/out_5 shows available m...,"Main Scenario:\nUser selects the ""Multimedia"" ...",None,None,None,2
9,25957,Mute/unmute the FM Radio playback,[I-25957] Mute/unmute the FM Radio playback\n...,Mute or pause function\nFunctional Description...,Description:,Preconditions:\nUser is playing FM Radio,"Goal: ""Users are able to mute/unmute the FM Ra...",Postconditions:\nThe user Mutes/Unmutes the FM...,Main Scenario:\n​​​User clicks on Mute/Unmute ...,None,None,None,3


# Save

In [12]:
ds.to_csv(data_dir / "ds" / "ds.unl", sep="|", encoding="utf-8", index=False)